In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [ ]:
model_name = "intfloat/e5-base-v2"
# BERT MODEL AND TOKENIZER
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name, output_hidden_states = True)

# MICROSOFT MODEL AND TOKENIZER
torch.random.manual_seed(0)
name2 = 'Qwen/Qwen2.5-1.5B-Instruct'
name1 = 'microsoft/Phi-3-mini-4k-instruct'
micro_model = AutoModelForCausalLM.from_pretrained(
    name2,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
micro_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 436210944 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
import os
import re
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

class VectorStore:
    def __init__(self, tokenizer, model):
        """
        Parameters:
        -----------

        tokenizer: Tokenizer (in this case BertTokenizer)
        model: Model (in this case BertModel)
        """
        self.tokenizer = tokenizer
        self.model = model
        self.vector_ids = []
        self.corpus = []

    def generate_embedding(self, text):
        """
        Takes a text and returns the embedding
        """
        inputs = self.tokenizer(text,
                                return_tensors='pt',
                                padding=True,
                                truncation= True,
                                max_length=512)

        with torch.no_grad():
            outputs = self.model(**inputs)

        return outputs.last_hidden_state[:, 0, :].numpy()

    def add_vector(self, texts: list[str]):
        """
        Creates embedding for list of text and adds it to a vector dictionary.
        Parameters:
        -----------
        texts -> list[str]: List of texts
        Returns:
        --------
        """

        for text in texts:
            embedding = self.generate_embedding(text).reshape(-1)
            self.vector_ids.append(embedding)
            self.corpus.append(text)


    def query_vector(self, query) -> str:
        embedding = self.generate_embedding(query)
        similarities = cosine_similarity(embedding, self.vector_ids)
        df = pd.DataFrame({"Text": self.corpus,
                "Similarities": similarities[0]})

        df.sort_values(by='Similarities', ascending=False, inplace=True, ignore_index=True)
        return ' '.join(df.loc[:5, "Text"])




def read_text_file(path: str) -> list[str]:
    """
    Read the text file

    Parameters:
    -----------

    path -> str: Path of text file

    Returns:
    ---------

    texts -> list[str]: Returns a list of sentences extracted from text file.
    """

    texts = []
    with open(path, 'r') as file:
        texts = file.readlines()

    return texts

DRIVE_PATH = '/content/drive/MyDrive/Colab Notebooks'

TEXT_PATH = os.path.join(DRIVE_PATH, 'Stock_related_definitions.txt')
texts = read_text_file(TEXT_PATH)

vector_store = VectorStore(tokenizer=bert_tokenizer,
                           model = bert_model)
vector_store.add_vector(texts=texts)
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
# pipe = pipeline(
#     task = "text-generation",
#     model=micro_model,
#     tokenizer=micro_tokenizer,
# )
while True:
    query = input("Chat here: ")
    if query == 'Q':
        print("Bye")
        break
    context = vector_store.query_vector(query)


    messages = [
        {
            "role": "system",
            "content": "You are a helpful AI assistant. "+context,
        },
        {
            "role": "user",
            "content": query},
    ]

    generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1,
    "do_sample": False,
    }
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # output = pipe(messages, **generation_args)
    # print(output[0]['generated_text'])
    pattern = r'<\|assistant\|>\s*([\s\S]*)'
    match = re.search(pattern, output[0]["generated_text"])

    if match:
        assistant_text = match.group(1).strip()
        print(assistant_text)
    else:
        print("No assistant text found.")




Chat here: What are dividends?
Dividends are payments made to shareholders by a company as a form of reward for their investment. Dividends are paid to shareholders based on the company's profits or earnings, which are determined through a process known as accounting. Dividends are paid out of the company's net earnings, which are the total amount of revenue earned by the company less any expenses.

Investors may receive dividends from the company's stock or other forms of ownership. Dividends can be paid out annually, quarterly, or semi-annually. When a company pays a dividend, it indicates that it has a positive net income and is willing to return part of that income to shareholders. Dividends are a form of compensation for shareholders, as they allow them to receive a share of the company's profits.
Chat here: i want to make money
Great! Here are some steps you can follow to make money as an AI assistant:

1. Create a bot: Create a chatbot using a bot builder like Dialogflow or Goog